In [1]:
import pandas as pd
import numpy as np
import censusdata
import openpyxl
import time
import re
import os
import sys

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [2]:
censusdata.__version__

'1.11'

##### do the preliminary work

* Open Excel file GIS_Census.xlsx  
<br>
* You are expected to change the data input. However, do not change column names and the order by which the parameters are displayed. Furthermore, do not write anything on the areas highlighted in gray.  
<br>
* In the "parameters tab", fill parameters according to your needs.  Nontheless, you must remember the following:  
    
    1. *If you are using "acs1" the summary level CANNOT be 140. "acs1" type is less granular, so 050 and 040 are the only levels available.*  
    <br>
    2. *You CANNOT mix certain table types, so you should download every type separately. For example, your "logic tab" CANNOT have at the same time the following attributes B25002_001E (a detail attribute from a detail table) and DP03_0001E (a detail attribute from a profile table) because the script will fail.*  
    <br>
    3. *Adjust parameter table according to the data type you are using. Options are ‘detail’ (detail tables), ‘subject’ (subject tables), ‘profile’ (data profile tables), ‘cprofile’ (comparison profile tables).*  
    <br>
    4. *Adjust your the "Naming Friendliness" parameter by using the most appropiate logic column name for your work.  For instance, a GIS analyst would be more interested in pulling data with "GIS Name" column names.  Options are ‘Schema Name’ (the census coded names), ‘Human Friendly Name’ (the human names assigned by the census bureau), ‘Mgr_Name’ (names assigned by the team of researchers), and ‘GIS_Name’ (names created by the GIS analyst to create a visualizations).*  
    <br>
<br>    
* In the "logic tab", fill parameters according to your needs.  Nontheless, you must remember the following:  

    1. *The "logic tab" MUST have NO data gaps.  This means that each schema name MUST have a human friendly name, mgr_name, a GIS_name and a description.  If you have no definition at hand, you should type down "No Definition".  This script uses this information not only to pull the data from the census bureau, but also to document the data search.  This script will delete ALL lines that are NOT complete.*
<br>
<br>
* You can pull data until 2011. However, you should be aware that the census bureau work is dynamic, which means that attributes that are tracked in 2019 are NOT necessarily available in previous year.  The script will throw an error message when the attribute does NOT exist.  
<br>
<br>


##### if  you are NOT sure about the table type you are dealing with    
    
The American Community Survey (ACS) provides four different sets of data tables:

* **Detailed tables**. These provide the most detailed set of variables. Table names begin with B followed by a numeric code.  

* **Data profile tables**. These tables are designed to provide information on a broad array of characteristics for a given geography. There are data profile tables on:  

    1. Social Characteristics (DP02),
    1. Economic Characteristics (DP03),
    1. Housing Characteristics (DP04), and
    1. Demographic Characteristics (DP05).  
    
  Table names (beginning in DP) are shown in parentheses above.  
  
 <br>

* **Subject tables**. These tables are designed to provide information on narrower topics for a broader range of geographies. Table names begin with S.

* **Comparison profile tables**. These tables provide information on changes in characteristics in particular geographies over time, including statistical significance testing. Table names begin with CP.
<br>
<br>


##### if  you are NOT sure about the summary level you are looking for 

This file only provides data for State, State-County and State-County-Census Tract.  To learn more bout the cartographic boundary file summary level, go visit:  
<br>
https://www.census.gov/programs-surveys/geography/technical-documentation/naming-convention/cartographic-boundary-file/carto-boundary-summary-level.html

##### set helper functions

In [3]:
def state_list(file_name, folder= 'inputs', sheetname = 'states', skiprows = 5):
    """
    This function creates a list of states numbers, which is used to process our
    our census query
    
    """
    
    # define path and data types
    full_path = os.path.join(folder, file_name)
    dtype = {'state_name': str, 'state_number': str}

    # read the states list from the census index
    state_list = pd.read_excel(full_path, sheet_name= sheetname, skiprows = skiprows, dtype = dtype)\
                              .dropna()    
    
    if state_list.shape[0] == 0:
        
        print("################################################################")
        print("")
        print("Number of rows: ", state_list.shape[0])
        print("")
        print("Please, enter at least a value in GIS Census file's tab 'states'")
        print("")
        print("################################################################")
    
    return state_list


def params_tupple( file_name, folder= 'inputs', sheetname = 'parameters', skiprows = 7):
    """
    This function stores the parameters used for the formater
    
    """
    
    # define path and data types
    full_path = os.path.join(folder, file_name)

    # read the states list from the census index
    params_tupple = pd.read_excel(full_path, sheet_name= sheetname, skiprows = skiprows)\
                              .dropna()    
    
    if (params_tupple.shape[1] <= 4) | params_tupple.shape[0] <= 1:
        
        #print(params_tupple)
        
        print("####################################################################")
        print("")
        print("####################  Parameter is missing  ########################")
        print("")
        print("Please, enter two parameters in GIS Census file's tab 'parameters'")
        print("")
        print("####################################################################")
        
        acs = None
        year = None
    
    else:
        #params_tupple
        acs = params_tupple.iloc[0,1].lower()
        year = params_tupple.iloc[1,1]
        sum_level = params_tupple.iloc[2,1]
        frly = params_tupple.iloc[3,1]
        table_type = params_tupple.iloc[4,1].lower()
    
    return acs, year, sum_level, frly, table_type


def logic_tab(file_name, folder= 'inputs', sheetname = 'logic', skiprows = 2):
    """
    This function procceses the data in the logic tab.  This data will be used to define the dictionaries
    needed to give human or GIS friendly names to the resulting metadata
    
    """
    
    # define path and data types
    full_path = os.path.join(folder, file_name)
    
    # read the states list from the census index
    logic_list = pd.read_excel(full_path, sheet_name= sheetname, skiprows = skiprows)\
                              .dropna()    
    
    if logic_list.shape[0] == 0:
        
        print("################################################################")
        print("")
        print("Number of rows: ", logic_list.shape[0])
        print("")
        print("Please, you must fill ALL the information requested in the logic tab.")
        print("\nOtherwise, no dataframe will be generated")
        print("")
        print("################################################################")
        
    # define list of fields that will be pulled from the census
    census_attributes = list(logic_list['Schema Name'])
    
    # create dictionaries to rename our dataframe's metadata
    human_dict = dict(zip(logic_list['Schema Name'], logic_list['Human Friendly Name']))
    mngr_dict = dict(zip(logic_list['Schema Name'], logic_list['Mgr_Name']))
    gis_dict = dict(zip(logic_list['Schema Name'], logic_list['GIS Name']))
    
    
    return logic_list, human_dict, mngr_dict, gis_dict, census_attributes


def tract_formatter(df, sum_level):
    """
    This function allows data analysts to get FIPS number out of the information provided by the given
    dataframe index
    
    """
    
    #reset index to find numbers by using regular expressions
    df.reset_index(inplace= True)
    df.rename(columns= {'index': 'summary'}, inplace= True)
    
    # we need to format tracts in a FIPS form, assign numbers to state, county and tracts, as well as
    # naming the states, counties, and tracts we are working with.  finally, we need to get rid of the columns
    # we do not need. the following dictionary will help to do all of this in a few lines of code.
    rename_dict = {140: {'tracts': {0:"census_tract_no", 1:"summary_level", 2:"state", 3:"county", 4:"tract"},
                         'splits': {1:"county_name", 2:"state_name"},
                         'cols':   [0, 3]},
                   
                   50: {'tracts': {0:"summary_level", 1:"state", 2:"county", 3:"tract"},
                        'splits': {0:"county_name", 1:"state_name"},
                        'cols':   [2]},
                   
                   40: {'tracts': {0:"summary_level", 1:"state"},
                        'splits': {0:"state_name"},
                        'cols':   [1, 2, 3]}}
    
    # extract the numbers assigned for summary level, state, county, tract and census tract
    tracts = df['summary'].apply(lambda x: pd.Series(re.findall(r'[0-9]+\.*[0-9]*',str(x)))
                                 .rename(rename_dict[sum_level]['tracts']))
        
     
    # with the information already extracted we can create our FIPS number by broadcasting the newly created
    # columns
    if sum_level == 140:
        tracts['census_tract'] = tracts['state'] + tracts['county'] + tracts['tract']
        
    elif sum_level == 50:
        tracts['census_tract'] = tracts['state'] + tracts['county']
        
    elif sum_level == 40:
        tracts['census_tract'] = tracts['state']
    
    
    # find human friendly names and drop columns you don't need
    if (sum_level == 140) | (sum_level == 50):
        splits = df['summary'].apply(lambda x: pd.Series(str(x).split(',')).rename(rename_dict[sum_level]['splits']))
        splits["state_name"] = pd.Series([i[0][:-1] for i in splits['state_name'].str.split()])
        
    else:
        splits = df['summary'].apply(lambda x: pd.Series(str(x).split(':')).rename(rename_dict[sum_level]['splits']))
    
    splits.drop(columns = rename_dict[sum_level]['cols'], inplace= True)
    
    # combine the new dataframe with the previous one
    df = pd.concat([df, tracts], axis= 1)
    df = pd.concat([df, splits], axis= 1)
    
    return df


def data_census_pull(acs, year, sum_level, list_of_states, table_type, census_attributes):
    """
    This function pulls data from the US census.  This allows to combine multiple states, which are pulled
    based on a specific summary level
    
    """
    # create dataframe that will store the resulting query
    census_df = pd.DataFrame()
    
    # let's doing our magic!! let's start pulling our census data
    for state in list_of_states:
        
        # set the geo finding per summary level
        census_geo_dict = {140: censusdata.censusgeo([('state', state), ('county', '*'), ('tract', '*')]),
                           50: censusdata.censusgeo([('state', state), ('county', '*')]),
                           40: censusdata.censusgeo([('state', state)])}
    
        # pull the data based on the summary level
        region_census = tract_formatter(censusdata.download(acs, year, census_geo_dict[sum_level],
                                                census_attributes, tabletype= table_type), sum_level)
        census_df = pd.concat([census_df, region_census])
          
            
    return census_df
    

def create_and_format_excel(acs, year, census_df, logic_list, sum_level):
    """
    This function creates an Excel file where the resulting dataframes will be pasted.
    Then, the data is formatted based on our color palette
    """
    
    ## INITIAL SET UP
    # create date object and output folder, if necessary
    today_date = time.strftime('%Y%m%d')
    today_date_display = time.strftime('%m-%d-%Y')
    
    if not os.path.exists('output'):
        os.mkdir('output')
    
    # create Excel file and its corresponding tabs
    excel_file_census = os.path.join('output','census_data_formatter_{}.xlsx'.format(today_date))
    writer = pd.ExcelWriter(excel_file_census, engine= 'xlsxwriter')
    workbook = writer.book
    
    worksheet1 = workbook.add_worksheet('census_data')
    worksheet2 = workbook.add_worksheet('logic')
    
    writer.sheets['census_data'] = worksheet1
    writer.sheets['logic'] = worksheet2
       
    ## FORMAT TABS
    # set formats for the dataframe.
    title_format = workbook.add_format({'bold': True,
                                        'italic': True,
                                        'font_size': 20, 
                                        'font_color': '#007377'})
    
    notes_format = workbook.add_format({'bold': True,
                                        'underline': True,
                                        'font_size': 13})
    
    comments_format = workbook.add_format({'italic': True,
                                          'font_size': 13})
    
    header_format = workbook.add_format({'bold': True,
                                        'text_wrap': True,
                                        'align': 'center',
                                        'valign': 'top',
                                        'font_size': 11,
                                        'font_color': '#e7e6e6',
                                        'fg_color': '#007377',
                                        'border': 1})
    
    # define columns width and comments for tab CENSUS_DATA
    worksheet1.set_column('A:A', 97)
    worksheet1.set_column('B:V', 17)
    worksheet1.set_column('W:Y', 22)
    
    worksheet1.write('A3', 'Census Data Extraction Outcome', title_format)
    worksheet1.write('A5', 'Notes:', notes_format)
    worksheet1.write('A6', 'The data pulled is from the American Community Survey ({}) for the year {}. Data pull date: {}'.format(acs.upper(), year, today_date_display), comments_format)
    worksheet1.write('A7', 'Summary Level: {}'.format(sum_level), comments_format)
    
    # Write the column headers with the defined format.
    for col_num, value in enumerate(census_df.columns.values):
        worksheet1.write(9, col_num, value, header_format)
    
    # define columns width and comments for tab LOGIC
    worksheet2.set_column('A:D', 37)
    worksheet2.set_column('E:E', 100)
    
    worksheet2.write('A3', 'Census Data Logic', title_format)
    worksheet2.write('A5', 'Notes:', notes_format)
    worksheet2.write('A6', 'The following details the equivalencies between the census columns and our naming convention', comments_format)
    worksheet2.write('A7', 'Summary Level: {}'.format(sum_level), comments_format)
    
    # Write the column headers with the defined format.
    for col_num, value in enumerate(logic_list.columns.values):
        worksheet2.write(9, col_num, value, header_format)
    
    ## PLACE DATAFRAMES AND FORMATS IN THE EXCEL OBJECT
    census_df.to_excel(writer, sheet_name= 'census_data', index= False, startrow = 10, startcol= 0, header= False)
    logic_list.to_excel(writer, sheet_name= 'logic', index= False, startrow = 10, startcol= 0, header= False)
    
    writer.save()

##### set parameters and create document

In [4]:

if __name__ == "__main__":
    
    ############################# PART 1: SET PARAMETERS ##################################
    
    # define the file name. this is the file where parameters, states and logic are stored
    file_name = 'GIS_Census_Index.xlsx'
    
    # read tabs 
    state_list = state_list(file_name)
    print('List of States: ', state_list)
    print('\n\n')
    
    acs, year, sum_level, frly, table_type = params_tupple(file_name)
    print('Extracting the following parameters: ')
    print('\n\n')
    print('ACS: ', acs)
    print('Year: ', year)
    print('Summary Level: ', sum_level)
    print('Frly: ', frly)
    print('Table Type: ', table_type)
    print('\n\n')
    
    logic_list, human_dict, mngr_dict, gis_dict, census_attributes = logic_tab(file_name) 
    #print('Logic List: ', logic_list)
    
    # exit the script in case some information is missing. parameters and geographies are 
    # mandatory to produce the information needed by the GIS team
    if (len(state_list) == 0) | (acs is None) | (year is None) | (logic_list.shape[0] == 0):
        
        sys.exit("Insuficient Information.  Please, review our Excel file and fill the gaps")
    
    
    ############################# PART 2: DATA PULL FROM CENSUS.ORG ########################
    
    # Define the list of states where the data will pulled. This will allow us to pull multiple states
    # at the same time.  Currently (version 1.0), only 3 summary levels are going to be brought
    list_of_states = list(state_list['state_number'])
    
    # create dataframe that will store the resulting query
    census_df = data_census_pull(acs, year, sum_level, list_of_states, table_type, census_attributes)
            
    # Our analyst will NOT be particularly interested in having a dataframe with coded names, so
    # we are changing the metadata
    cols_dict = {'Schema Name': census_attributes,
                 'Human Friendly Name': human_dict,
                 'Mgr_Name': mngr_dict,
                 'GIS Name': gis_dict}
    
    
    try:
        census_df.rename(columns= cols_dict[frly], inplace= True)
    except:
        pass
        
    
    ############################# PART 3: CREATE EXCEL FILE FOR R&E ########################
    create_and_format_excel(acs, year, census_df, logic_list, sum_level)
    
    print("Document created")
    



List of States:    state_name state_number
0   Illinois           17



Extracting the following parameters: 



ACS:  acs5
Year:  2019
Summary Level:  140
Frly:  GIS Name
Table Type:  detail



Document created
